In [3]:
#作业编写程序  增量备份

import os
import filecmp
import shutil
import sys
'''
这里使用了2个新的模块 filecmp shutil
filecmp提供了3个操作的方法，cmp（单文件对比） cmpfile（多文件对比） dircmp（目录对比）

shutil：该模块拥有许多的文件夹操作的功能，包括复制，移动重命名，删除等等
'''
def usage():
    print('源目录和目标必须是已存在的绝对路径表示')
    print('语法格式，例如：{0} c:\\oldir c:\\newdir'.format(sys.argv[0] ) )
    sys.exit(0)
    
def autoBackup(scrDir,dsDir):
    #判断输入的参数是否合理
    if ((not os.path.isdir(scrDir))or(not os.path.isdir(dstDir))or
       (os.path.abspath(scrDir)!=scrDir) or (os.path.abspath(dstDir)!=dstDir)):
        usage()
    for item in os.listdir(scrDir):
        scrItem = os.path.join(scrDir,item)
        dstItem = scrItem.replace(scrDir,dstDir)
        if os.path.isdir(scrItem):
            #创建新的文件夹，保证与源文件夹一致
            if not os.path.exists(dstItem):
                os.makedirs(dstItem)
                print('创建目录'+dstItem)
            #递归向下创建
            autoBackup(scrItem,dstItem)
        elif os.path.isfile(scrItem): #如果是文件
            #只复制新增或修改的文件
            if ((not os.path.exisits(dstItem)) or(not filecmp.cmp(scrItem,dstItem,shallow=False))):
                shutil.copyfile(scrItem,dstItem)
                print('file:'+scrItem+'==>'+dstItem)
                
if __name__=='__main__':
    scrDir,dstDir='/data','/datadestination'
    autoBackup(scrDir,dstDir)

源目录和目标必须是已存在的绝对路径表示
语法格式，例如：C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py c:\oldir c:\newdir


SystemExit: 0

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2870: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
#作业四： 递归删除指定文件夹中指定类型的文件
from os.path import isdir,join,splitext
from os import  remove ,listdir,getcwd

filetypes= ['rtf']  #待删除的文件类型列表
def delCertainFiles(directory):
    for filename in listdir(directory):
        temp = join(directory,filename)
        if isdir(temp):
            delCertainFiles(temp)
        elif splitext(temp)[1][1:] in filetypes:   #分割路径名与扩展名并把 . 用切片方法消除
            remove(temp)
            print(temp,'deleted.....')
        
def main():
    print('待删除的文件列表：',filetypes)
    print('当前的工作目录：'getcwd())
    delCertainFiles( getcwd() )

main()

In [7]:
''' 
学生成绩：假设某学校所有课程每学期允许多次考试，学生可以随时参加考试，系统将每次的考试
成绩自动的添加到excel中 （有3列  姓名  课程 成绩） 现在期末要求统计学生每门课程的最高成绩
'''
import openpyxl
from openpyxl import Workbook
import random
from os import remove,listdir,getcwd
from os.path import isdir,join,splitext

#生成随机数据
def generateRandomInformation(filename):
    workbook=Workbook()
    worksheet=workbook.worksheets[0]
    worksheet.append(['姓名','课程','成绩'])
    
    #中文名字的第一个 第二个 第三个 字
    first='赵钱孙李'
    middle='伟呁琛东'
    last='坤哲思'
    subjects=('语文','数学','英语')
    for i in range(200):
        line=[]
        r = random.randint(1,100)
        name=random.choice(first)
        #有2个字的名字
        if r>50:
            name=name+ random.choice(middle)
        name=name+random.choice(last)
        #一次生成姓名 课程  分数
        line.append(name)
        line.append(random.choice(subjects) )
        line.append(random.randint(0,100))
        
        worksheet.append(line)
     #保存数据  生成Excel 格式的文件
    workbook.save(filename)
    
def getResult(oldfile,newfile):
    #用于存放结果的数据的字典
    result=dict()
    #打开原始的数据
    workbook=openpyxl.load_workbook(oldfile)
    worksheet = workbook.worksheets[0]
    
    #遍历原始数据
    for row in worksheet.rows:
        if row[0].value == '姓名':
            continue
        #姓名 课程 成绩
        name,subject,grade=row[0].value,row[1].value,row[2].value
        
        #获取当前姓名对应的课程名称和成绩信息
        t=result.get(name,{})
        #获取当前学生当前课程的成绩，若不存在返回0
        f=t.get(subject,0)
        #只保留该学生最高的成绩
        if grade > f:
            t[subject] =grade
            result[name]=t
    workbook1=Workbook()
    worksheet1=workbook1.worksheets[0]
    worksheet1.append(['姓名','课程','成绩'])
    
    #将result字典中的结果数据写入Excel文件
    for name,t in result.items():
        print(name,t)
        for subject,grade in t.items():
            worksheet1.append([name,subject,grade])
            
    workbook1.save(newfile)
    
if __name__=='__main__':
    oldfile= join(getcwd(),r'test.xlsx')
    newfile=join(getcwd(),r'result.xlsx')
    
    generateRandomInformation(oldfile)
    getResult(oldfile,newfile)

李伟思 {'英语': 45, '数学': 95, '语文': 7}
孙呁坤 {'语文': 15, '数学': 50}
钱呁坤 {'数学': 71, '语文': 86}
钱伟哲 {'数学': 75}
孙坤 {'英语': 77, '语文': 75, '数学': 83}
李坤 {'数学': 72, '语文': 97, '英语': 84}
孙思 {'英语': 42, '语文': 84, '数学': 97}
钱东思 {'数学': 87}
钱坤 {'语文': 71, '英语': 87, '数学': 60}
李呁坤 {'数学': 12, '语文': 31, '英语': 11}
孙哲 {'英语': 55, '语文': 83, '数学': 73}
赵呁哲 {'数学': 17, '英语': 56}
赵伟哲 {'英语': 33, '语文': 69}
李琛坤 {'英语': 100}
赵伟思 {'语文': 54, '英语': 51}
李东哲 {'语文': 88}
钱东坤 {'英语': 43}
赵坤 {'数学': 42, '语文': 54, '英语': 22}
孙伟思 {'英语': 91, '语文': 12, '数学': 48}
钱呁思 {'英语': 93}
孙呁思 {'英语': 76, '数学': 76}
李哲 {'英语': 90, '语文': 88, '数学': 85}
赵哲 {'英语': 10, '数学': 55, '语文': 94}
赵琛思 {'英语': 23, '语文': 78, '数学': 82}
钱哲 {'英语': 88, '数学': 63, '语文': 19}
孙伟坤 {'语文': 86, '数学': 2}
赵思 {'数学': 74, '英语': 93, '语文': 55}
钱琛思 {'数学': 81, '语文': 50}
赵琛哲 {'英语': 27, '数学': 78}
孙琛思 {'英语': 12, '数学': 39}
赵呁坤 {'英语': 56, '数学': 71}
钱呁哲 {'语文': 7, '英语': 98}
李思 {'语文': 96, '数学': 91, '英语': 53}
李呁思 {'数学': 57, '语文': 7}
钱思 {'语文': 81, '数学': 41}
孙琛坤 {'语文': 96, '英语': 33}
钱伟思 {'英语': 63, '数学': 84}


In [ ]:
#记事本 test.txt 转换成excel test.txt文件中的第一行作为开头 第二行开始时实际数据 并且表头与数据中的不同的字段信息都是用逗号分隔
from openpyxl import Workbook

def main(txtFileName):
    new_XlsxFileName = txtFileName[:-3]+'xlsx'
    wb=Workbook()
    ws=wb.worksheets[0]
    with open(txtFileName) as fp:
        for line in fp:
            line = line.strip().split(',')
            ws.append(line)
    wb.save(new_XlsxFileName)
    
main('test.txt')

In [9]:
#电影演员对应表
import openpyxl
from openpyxl import Workbook
def getActors(filename):
    actors=dict()
    #打开xlsx文件  并获取第一个worksheet
    wb=openpyxl.load_workbook(filename)
    ws=wb.worksheets[0]
    #遍历Excel文件中的所有行
    for index,row in enumerate(ws.rows):
        #绕过第一行的开头
        if index==0:
            continue
        #获取电影名称与演员列表
        filmName,actor=row[0].value,row[2].value.split(',')
        for a in actor:
            actors[a]=actors.get(a,set())
            actors[a].add(filmName)
    return actors

actors=getActors('电影导演演员.xlsx')
actors=sorted(actors.items(),key=lambda x:int(x[0][2:]))
for item in actors:
    print(item)

In [1]:
#文件的重复判定
import hashlib

fo1=open('a.png','rb')
content1=fo1.read()
fo2=open('b.png','rb')
content2=fo2.read()

md51=hashlib.md5( content).hexdigest()
md52=hashlib.md5( content).hexdigest()

print(md51)
print(md52)

fo1.close()
fo2.close()

FileNotFoundError: [Errno 2] No such file or directory: 'a.png'

In [5]:
import os
import time
basedir = '/root/script'
filelists = []
# 指定想要统计的文件类型
whitelist = ['java']
#遍历文件, 递归遍历文件夹中的所有
def getFile(basedir):
    global filelists
    for parent,dirnames,filenames in os.walk(basedir):
        #for dirname in dirnames:
        #    getFile(os.path.join(parent,dirname)) #递归
        for filename in filenames:
            ext = filename.split('.')[-1]
            #只统计指定的文件类型，略过一些log和cache文件
            if ext in whitelist:
                filelists.append(os.path.join(parent,filename))
#统计一个文件的行数
def countLine(fname):
    count = 0
    for file_line in open(fname).xreadlines():
        if file_line != '' and file_line != '\n': #过滤掉空行
            count += 1
    print (fname + '----' , count)
    return count

if __name__ == '__main__' :
    startTime = time.clock()
    getFile(basedir)
    totalline = 0
    for filelist in filelists:
        totalline = totalline + countLine(filelist)
    print ('total lines:',totalline)

total lines: 0
